In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Load your dataset
data = pd.read_csv("mlb_1998_2024.csv")

# Create derived columns
data["win_pct"] = data["wins"] / (data["wins"] + data["losses"])
data["run_diff"] = data["runs_scored"] - data["runs_allowed"]

# Output folder for HTML files
output_folder = "reports"
os.makedirs(output_folder, exist_ok=True)

# Get list of teams
teams = sorted(data["team_name"].unique())

# Compute league average wins by year
league_avg = data.groupby("year")["wins"].mean().reset_index()

# Generate one interactive HTML report per team
for team in teams:
    team_data = data[data["team_name"] == team].sort_values("year")

    fig = make_subplots(rows=2, cols=2, subplot_titles=[
        "Wins by Season",
        "Run Differential by Season",
        "Win % by Season",
        "Wins vs League Average"
    ])

    # Wins
    fig.add_trace(go.Scatter(x=team_data["year"], y=team_data["wins"],
                             mode="lines+markers", name="Wins", line=dict(color="navy")),
                  row=1, col=1)

    # Run Differential
    fig.add_trace(go.Scatter(x=team_data["year"], y=team_data["run_diff"],
                             mode="lines+markers", name="Run Diff", line=dict(color="crimson")),
                  row=1, col=2)

    # Win Percentage
    fig.add_trace(go.Scatter(x=team_data["year"], y=team_data["win_pct"],
                             mode="lines+markers", name="Win %", line=dict(color="teal")),
                  row=2, col=1)

    # Wins vs League Avg
    fig.add_trace(go.Scatter(x=team_data["year"], y=team_data["wins"],
                             mode="lines+markers", name=team, line=dict(color="blue")),
                  row=2, col=2)
    fig.add_trace(go.Scatter(x=league_avg["year"], y=league_avg["wins"],
                             mode="lines", name="League Avg", line=dict(color="gray", dash="dot")),
                  row=2, col=2)

    fig.update_layout(height=800, width=1000, title_text=f"{team} Report (1998–2024)")

    filename = team.replace(" ", "_") + ".html"
    fig.write_html(os.path.join(output_folder, filename))

print("All reports generated in /reports/")
